<a href="https://colab.research.google.com/github/gragbag/NestedFunctionSyntaxChecker/blob/main/cs3110_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import re

class PDA:
  def __init__(self):
    self.stack = ['Z']
    self.state = 'q0'
    self.path = []
    self.terminals = {'int', 'float', 'void', '=', ',', '+', '*', '{', '}', '(', ')', ';', 'return'}
    self.transitions = {
        ('q0', None, 'Z'): [('q1', 'FuncDecl Z'), ('q2', 'Type Assign Z')],

        #q1 FuncDecl
        ('q1', None, 'FuncDecl'): [('q1', 'ReturnType FunctionName ( ParamList ) Block')],

        #q1 ReturnType
        ('q1', 'int', 'ReturnType'): [('q1', None)],
        ('q1', 'float', 'ReturnType'): [('q1', None)],
        ('q1', 'void', 'ReturnType'): [('q1', None)],

        #q2 Type
        ('q2', 'int', 'Type'): [('q7', None)],
        ('q2', 'float', 'Type'): [('q7', None)],

        #q3 ParamList
        ('q1', 'var', 'FunctionName'): [('q3', None)],

        ('q3', '(', '('): [('q3', None)],
        ('q3', ')', ')'): [('q3', None)],
        ('q3', None, 'ParamList'): [('q4', 'Param'), ('q4', 'Param , ParamList')],
        ('q3', ',', ','): [('q3', None)],

        #q4 Params
        ('q4', 'int', 'Param'): [('q3', 'var')],
        ('q4', 'float', 'Param'): [('q3', 'var')],
        ('q3', 'var', 'var'): [('q3', None)],

        #q3 Block, q5 StatementList
        ('q3', '{', 'Block'): [('q5', 'StmtList }')],

        ('q5', None, 'StmtList'): [('q6', 'Stmt ;'), ('q6', 'Stmt ; StmtList')],
        ('q5', '}', '}'): [('q0', None)], #When the function declaration ends, go back to q0
        ('q6', ';', ';'): [('q5', None)],
        ('q6', ';', 'Z'): [('q0', 'Z')], #When the assignment finishes, go back to z0
        ('q6', ')', ')'): [('q6', None)],
        ('q6', '+', 'Operator'): [('q9', 'Expr')],
        ('q6', '*', 'Operator'): [('q9', 'Expr')],

        #q6 statements
        ('q6', None, 'Stmt'): [('q7', 'Assign'), ('q8', 'Return')],

        #q7 assign
        ('q7', 'var', 'Assign'): [('q9', '= Expr')],

        #q9 Expr and Operator
        ('q9', '=', '='): [('q9', None)],

        ('q9', 'var', 'Expr'): [('q6', None), ('q9', 'Operator'), ('q13', '( ArgList )'), ('q13', '( ArgList ) Operator')],
        ('q9', 'num', 'Expr'): [('q6', None), ('q9', 'Operator')],
        ('q9', ';', ';'): [('q5', None)],
        ('q9', ',', ','): [('q13', None)],

        ('q9', '+', 'Operator'): [('q9', 'Expr')],
        ('q9', '*', 'Operator'): [('q9', 'Expr')],

        #q11 FuncCall, q13 ArgList
        ('q11', 'var', 'FuncCall'): [('q13', '( ArgList )')],
        ('q13', '(', '('): [('q13', None)],
        ('q13', ')', ')'): [('q9', None)],
        ('q13', None, 'ArgList'): [('q9', 'Expr'), ('q9', 'Expr , ArgList')],

        #q8 Return
        ('q8', 'return', 'Return'): [('q9', 'Expr')],
    }


  def get_possible_transitions(self, symbol, top_of_stack):
    return self.transitions.get((self.state, symbol, top_of_stack))

  def push_to_stack(self, variables):
    if variables:
      for var in reversed(variables.split()):
          self.stack.append(var)

  def classify_token(self, token):
    if token in self.terminals:
      return token
    elif is_number(token):
      return 'num'
    else:
      return 'var'


  def process_input(self, tokens):
    # Recursive backtracking helper function
    def backtrack(index, step):
      if index == len(tokens):
        # If we've processed all tokens and stack is at the initial state, return True
        return self.stack[-1] == 'Z' and self.state == 'q0'

      token = tokens[index]
      print(f"Stack: {self.stack}, Input: {token}, State: {self.state}, Top: {self.stack[-1]}")


      top_of_stack = self.stack.pop()
      curr_state = self.state
      curr_stack = self.stack.copy()
      curr_path = self.path.copy()

      #Include as token, which can be a keyword, var, or num
      token_type = self.classify_token(token) #Returns token, num, or var
      possible_transitions = self.get_possible_transitions(token_type, top_of_stack)

      if possible_transitions:

        for next_state, variables in possible_transitions:
          print("possible: ", possible_transitions)
          print("Now Trying: ", variables)
          self.path.append((step, self.state, token, top_of_stack, next_state, variables))
          self.state = next_state

          self.push_to_stack(variables)

          if backtrack(index + 1, step + 1):
            return True

          self.state = curr_state
          self.stack = curr_stack.copy()
          self.path.pop()

      #Do not include token
      possible_transitions = self.get_possible_transitions(None, top_of_stack)
      if possible_transitions:

        for next_state, variables in possible_transitions:
          print("possible from None: ", possible_transitions)
          print("Now Trying From None: ", variables)
          self.path.append((step, self.state, 'None', top_of_stack, next_state, variables))
          self.state = next_state

          self.push_to_stack(variables)

          if backtrack(index, step + 1):
            return True

          self.state = curr_state
          self.stack = curr_stack.copy()
          self.path.pop()

      return False


    return backtrack(0, 0)

  def display_path(self):
    print("Transition Path:")
    for (step, current_state, input_symbol, stack_top, next_state, stack_op) in self.path:
        print(f"Step {step}: From state {current_state}, input '{input_symbol}', stack top '{stack_top}' -> "
              f"Next state {next_state}, stack push: '{stack_op}'")

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [17]:
def get_tokens(program):
  token_regex = r'\bint\b|\bfloat\b|\bvoid\b|\breturn\b|\b[a-zA-Z_]\w*\b|\d+[.]?\d*|[,{}();+=*]'
  tokens = re.findall(token_regex, program)
  return tokens


pda = PDA()
# program = """
# int f(int a) {
#   return a + 1;
# }

# int g(int b) {
#   return f(b) * 2;
# }

# int h(int c) {
#   return g(f(c));
# }

# int result = h(5);
# """

program = """
int f(int a) {
  return a + 1;
}

int g(int b) {
  return f(b) * 2;
}

int h(int c) {
  return g(f(c));
}

int result = h(5);
"""

tokens = get_tokens(program)

if pda.process_input(tokens):
  print("The program syntax is valid!")
  pda.display_path()
else:
  print("The program syntax is invalid!")

Stack: ['Z'], Input: int, State: q0, Top: Z
possible from None:  [('q1', 'FuncDecl Z'), ('q2', 'Type Assign Z')]
Now Trying From None:  FuncDecl Z
Stack: ['Z', 'FuncDecl'], Input: int, State: q1, Top: FuncDecl
possible from None:  [('q1', 'ReturnType FunctionName ( ParamList ) Block')]
Now Trying From None:  ReturnType FunctionName ( ParamList ) Block
Stack: ['Z', 'Block', ')', 'ParamList', '(', 'FunctionName', 'ReturnType'], Input: int, State: q1, Top: ReturnType
possible:  [('q1', None)]
Now Trying:  None
Stack: ['Z', 'Block', ')', 'ParamList', '(', 'FunctionName'], Input: f, State: q1, Top: FunctionName
possible:  [('q3', None)]
Now Trying:  None
Stack: ['Z', 'Block', ')', 'ParamList', '('], Input: (, State: q3, Top: (
possible:  [('q3', None)]
Now Trying:  None
Stack: ['Z', 'Block', ')', 'ParamList'], Input: int, State: q3, Top: ParamList
possible from None:  [('q4', 'Param'), ('q4', 'Param , ParamList')]
Now Trying From None:  Param
Stack: ['Z', 'Block', ')', 'Param'], Input: int,